In [3]:
%pip install numpy

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/16/2e/86f24451c2d530c88daf997cb8d6ac622c1d40d19f5a031ed68a4b73a374/numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.0 kB 222.6 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.0 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 362.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
    --------------------------------------- 0.2/15.5 MB 6.3 MB/s eta 0:00:03
   - -------------------------------------- 0.6/15.5 MB 7.5 MB/s eta 0:00:02
   -- ------------------------------------


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/22/a5/a0b255295406ed54269814bc93723cfd1a0da63fb9aaf99e1364f07923e5/pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b6a0ee9fb87e6e7a1f6bcab2cdc73f02bb7ba91ada0/tzdata-2024.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
    --------------------------------------- 0.2/11.5 MB 5.0 MB/s eta 0:00:03
   -- ------------------------------------- 0.8/11.5 MB 8.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/11.5 MB 11.0 MB/s eta 0:00:0


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
%pip install networkx

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for networkx from https://files.pythonhosted.org/packages/38/e9/5f72929373e1a0e8d142a130f3f97e6ff920070f87f91c4e13e40e0fba5a/networkx-3.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
    --------------------------------------- 0.0/1.7 MB 991.0 kB/s eta 0:00:02
   --------------- ------------------------ 0.7/1.7 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.7 MB 13.5 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 12.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/17/91/febbb6c1063ae05a62fdbe038c2917b348b1b35f0482cee4738e6870a44a/matplotlib-3.9.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/78/38/a046bb0ebce6f530175d434e7364149e338ffe1069ee286ed8ba7f6481ee/contourpy-1.2.1-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/79/3e/3d87230c46a103ae225d47ae59ac6cdbd4fa341dddaffdfaf4e89986dd88/fonttools-4.53.0-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/165.5 kB ? eta


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt 

# Base para todos

In [7]:
def exp_mechanism_geral(data, score_func, list_outputs, epsilon, global_sensivity, target = None):
    scores = [score_func(data, out, target) for out in list_outputs]
    probs = [np.exp(epsilon * score / (2 * global_sensivity)) for score in scores]
    probs_normalizadas = probs / np.linalg.norm(probs, ord=1)

    return np.random.choice(list_outputs, 1, p=probs_normalizadas)[0]

def dampening_function(data, output, score_func, delta_func):
    def b_func(data, i, output):
        if i == 0:
            return 0
        return sum([delta_func(data, j, output) for j in range(i - 1)])
    
    i = 0  
    score = score_func(data, output)
    while True:
        b_i = b_func(data, i, output)
        b_ipp = b_func(data, i + 1, output)
        if (score > 0 and (score < b_i or score >= b_ipp)) or (score < 0 and (score < -b_ipp or score >= -b_i)):
            i += 1
            continue

        return (score - b_i) / (b_ipp - b_i) + i if score > 0 else (score + b_i) / (-b_ipp + b_i) - i

def local_dampening_mechanism(data, epsilon, score_func, delta_func, list_outputs):
    scores = [dampening_function(data, out, score_func, delta_func) for out in list_outputs]
    probs = [np.exp(epsilon * score / 2) for score in scores]
    probs_normalizadas = probs / np.linalg.norm(probs, ord=1)

    return np.random.choice(list_outputs, 1, p=probs_normalizadas)[0]


def permute_and_flip(data, epsilon, score_func, list_outputs, max_u):
    max_score = max([score_func(data, out) for out in list_outputs])
    # aqui talvez seja interessante copiar e passar a copia da list_ouputs
    np.random.shuffle(list_outputs)
    for r in list_outputs:
        prob = np.exp(epsilon * (score_func(data, r) - max_score) / (2 * max_u))
        if np.random.rand() < prob:
            return r


# Algoritmo 7

In [28]:
data = {
 "Outlook": ["rain", "rain", "sun", "sun", "sun"], 
 "Wind": ["strong", "weak", "strong", "weak", "strong"], 
 "Decision":  ["Y", "N", "Y", "N", "N"]
}

df = pd.DataFrame(data)
df

,Outlook,Wind,Decision
0,rain,strong,Y
1,rain,weak,N
2,sun,strong,Y
3,sun,weak,N
4,sun,strong,N


In [29]:
def get_canditates(data, t, value_column, target, coluna, label):
    # se a distancia for 0
    if t == 0:
      # dataframe onde o atributo A possui valor j
      ## no caso, a coluna possui value_column
      qtd_T_value_columnA = data[data[coluna] == value_column]
      # dataframe onde o atributo A possui valor j e alvo é c de C
      ## no caso, a coluna possui value_column e o label é oque procuramos
      value_columnA_target = qtd_T_value_columnA[qtd_T_value_columnA[label] == target]

      return qtd_T_value_columnA.shape[0], value_columnA_target.shape[0]
    candidates = get_canditates(data, t-1, value_column, target, coluna, label)
    validos_candidates = []

    for x, y in candidates:
      if x > 0 and y > 0:
        validos_candidates.append((x - 1, y - 1))
      if x < data.shape[0]:
        validos_candidates.append((x + 1, y))
    return validos_candidates

def h_func(x, y):
    def f_func(x, somar = False):
        return x * np.log2((x + 1) / x) + np.log2(x + 1) if somar else x * np.log2((x - 1) / x) - np.log2(x - 1)
    return max(f_func(x, True) - f_func(y, True), f_func(x) - f_func(y))

def local_sensivity_at_tDistance(data:pd.DataFrame, t:float, col_val, target):
    '''
    data = dataframe

    t = distâncuia

    col_val = uma coluna do dataframe

    target_values = lista com todas saídas possíveis do label
    '''
    H = []
    for a in data[col_val].unique():
      # iterar sobre as classes daquela coluna
      for c in data[target].unique():
        # ver os candidatos
        candidates = get_canditates(data, t, a, c, col_val, target)
        # para cada candidato, calcular a h_func e add a H
        for x, y in candidates:
          H.append(h_func(x,y))
    # retoanr o maior valor H (entropy)
    return max(H)

In [30]:
def noisy_count(data: pd.DataFrame):
    pass

# Equação 7.2
def score_func(data: pd.DataFrame, r, target):
    '''
    r = uma coluna específica do dataframe
    '''
    infGain_list = []
    for c in data[target].unique():
      for a in data[r].unique():
        data_colR_valA = data[data[r] == a]
        data_colR_valA_outC = data_colR_valA[data_colR_valA[target] == c]
        tau_jc = data_colR_valA_outC.shape[0]
        tau_j = data_colR_valA.shape[0]
        infGain_list.append(tau_jc * np.log2(tau_jc / tau_j))

    return -sum(infGain_list)

def build_diff_PID3(data:pd.DataFrame, list_columns: list, target, profundidade, epsilon):
    t_val = max([len(data[col].unique()) for col in list_columns])
    N_t = noisy_count()
    data_target_list = data[target].unique()
    tree = nx.DiGraph()
    if len(list_columns) == 0 or profundidade == 0 or N_t / (t_val * len(data_target_list)) < np.sqrt(2)/2:
      data_output_partitions = [noisy_count(data[data[target] == c]) for c in data_target_list]
      max_Nc = max(data_output_partitions)
      tree.add_node(max_Nc)
      return tree
    # columns_hat = local_dampening_mechanism(data, epsilon, ig_func, local_sensivity_at_tDistance, list_columns)
    columns_hat = exp_mechanism_geral(data, epsilon, score_func, list_columns)
    data_i = [data[data[columns_hat] == i] for i in data[columns_hat].unique()]
    
    for i in data_i:
      sub_tree_i = build_diff_PID3(i, list_columns.remove(columns_hat), target, profundidade - 1, epsilon)
      tree.add_edge(columns_hat, sub_tree_i, label = f"{columns_hat}->{i}")
    
    return tree

def global_diff_PID3(data:pd.DataFrame, target, profundidade: int, privacy_budget: float):
    '''
    data = um dataframe
    target = a coluna alvo, no exemplo do tempo, seria o Y e o N
    profundidade = numero que indica a profuntidade
    privacy_budget = nivel de privacidade numero
    '''
    epsilon = privacy_budget / (2 * (profundidade + 1))
    lista_colunas = data.columns.to_list()
    lista_colunas.remove(target)
    return build_diff_PID3(data, lista_colunas, target, profundidade, epsilon)

arvore = global_diff_PID3(df, "Decision", 2, 0.5)

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'